## Latest NMT Model Attempt following a google translation model

In [1]:
from transformers import BertTokenizer, TFBertModel, BertGenerationEncoder, BertGenerationDecoder
import pandas as pd
import numpy as np
from nltk.translate import bleu
import tensorflow as tf
from transformers import Trainer, TrainingArguments, Seq2SeqTrainer, EncoderDecoderModel, BertTokenizer, EncoderDecoderConfig, BertConfig, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, default_data_collator
import pandas as pd
import numpy as np
from nltk.translate import bleu
import tensorflow as tf
import torch
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split

In [6]:
data = load_dataset('json', data_files = 'thai_web_train.json')
data

Using custom data configuration default-34b83d8f1977e07e


Dataset json downloaded and prepared to /home/akihiczak/.cache/huggingface/datasets/json/default-34b83d8f1977e07e/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 10000
    })
})

In [2]:
d = pd.read_csv("scb-mt-en-th-2020/apdf.csv")

In [3]:
metric = load_metric("sacrebleu")

In [4]:
datasets = load_dataset('csv', data_files='scb-mt-en-th-2020/apdf.csv')

Using custom data configuration default-3a2c74c780365b63
Reusing dataset csv (/home/akihiczak/.cache/huggingface/datasets/csv/default-3a2c74c780365b63/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['en_text', 'th_text'],
        num_rows: 13503
    })
})

In [6]:
train_dataset = datasets["train"]
train_dataset

Dataset({
    features: ['en_text', 'th_text'],
    num_rows: 13503
})

In [7]:
test_dataset = train_dataset.select(range(13499, 13503))
test_dataset

Dataset({
    features: ['en_text', 'th_text'],
    num_rows: 4
})

In [8]:
train_dataset = train_dataset.select(range(1500))
#train_dataset = train_dataset.select(range(32))
train_dataset

Dataset({
    features: ['en_text', 'th_text'],
    num_rows: 1500
})

In [9]:
encoder = BertGenerationEncoder.from_pretrained("bert-base-multilingual-cased", is_decoder = False, is_encoder_decoder = False)
encoder

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertGenerationEncoder: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'bert.embeddings.token_type_embeddings.weight']
- This IS expected if you are initializing BertGenerationEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertGenerationEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertGenerationEncoder(
  (embeddings): BertGenerationEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (int

In [10]:
decoder = BertGenerationDecoder.from_pretrained("bert-base-multilingual-cased", is_decoder = True, add_cross_attention = True, is_encoder_decoder = False)
decoder

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertGenerationDecoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'bert.embeddings.token_type_embeddings.weight']
- This IS expected if you are initializing BertGenerationDecoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertGenerationDecoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some w

BertGenerationDecoder(
  (bert): BertGenerationEncoder(
    (embeddings): BertGenerationEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (d

In [11]:
model = EncoderDecoderModel(encoder=encoder, decoder=decoder)
model.config

EncoderDecoderConfig {
  "decoder": {
    "_name_or_path": "bert-base-multilingual-cased",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "bos_token_id": 2,
    "chunk_size_feed_forward": 0,
    "decoder_start_token_id": null,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 1,
    "finetuning_task": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "is_decoder": true,
    "is_encoder_decoder": false,
    "label2id": {
      "LABEL_0": 0,
      "LABEL_1": 1
    },
    "layer_norm_eps": 1e-12,
    "length_penalty": 1.0,
    "max_length": 20,
    "max_

In [12]:
model.config.decoder.architectures = ["BertGenerationDecoder"]

In [13]:
model.config.encoder.architectures = ["BertGenerationEncoder"]

In [14]:
model.config.decoder.decoder_start_token_id = 2

In [15]:
model.config.encoder.tokenizer_class = "BertTokenizer"

In [16]:
model.config.decoder.tokenizer_class = "BertTokenizer"

In [17]:
model.config

EncoderDecoderConfig {
  "decoder": {
    "_name_or_path": "bert-base-multilingual-cased",
    "add_cross_attention": true,
    "architectures": [
      "BertGenerationDecoder"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "bos_token_id": 2,
    "chunk_size_feed_forward": 0,
    "decoder_start_token_id": 2,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 1,
    "finetuning_task": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "is_decoder": true,
    "is_encoder_decoder": false,
    "label2id": {
      "LABEL_0": 0,
      "LABEL_1": 1
    },
    "layer_norm_eps": 1e-12,
    "length_penalty": 1.0,
    "max_length": 20,
    "m

In [18]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
#model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')
#model.config.output_attentions = True

In [19]:
#encoder2 = BertGenerationEncoder.from_pretrained("bert-base-multilingual-cased", bos_token_id=101, eos_token_id=102)
# add cross attention layers and use BERT's cls token as BOS token and sep token as EOS token
#decoder2 = BertGenerationDecoder.from_pretrained("bert-base-multilingual-cased", add_cross_attention=True, is_decoder=True, bos_token_id=101, eos_token_id=102)
#model_old = EncoderDecoderModel(encoder=encoder2, decoder=decoder2)
#model_old.config

In [9]:
# create tokenizer...
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [10]:
column_names = datasets["train"].column_names

In [11]:
max_source_length = max([len(tokenizer(x, return_tensors='pt', truncation=True)["input_ids"][0]) for x in d['th_text']])
max_target_length = max([len(tokenizer(x, return_tensors='pt', truncation=True)["input_ids"][0]) for x in d['en_text']])

In [12]:
max_target_length

365

In [13]:
def preprocess_function(examples):
        inputs = [ex for ex in examples["th_text"]]
        targets = [ex for ex in examples["en_text"]]
        model_inputs = tokenizer(inputs, max_length=max_source_length, padding="max_length", truncation=True)

        # Setup the tokenizer for targets
        decoder = tokenizer(targets, return_tensors='pt',max_length=max_target_length, padding="max_length", truncation=True)
    
        decoder_ids = decoder["input_ids"]
        decoder_ids = decoder_ids[:, 1:]
        zeros = torch.zeros((len(examples["en_text"]), 1), dtype=torch.long)
        decoder_ids = torch.cat((decoder_ids, zeros), 1)

        # Was Label Previously
        model_inputs["decoder_input_ids"] = decoder["input_ids"].tolist()
        model_inputs["labels"] = decoder_ids.tolist()

        return model_inputs

In [14]:
train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            remove_columns=column_names,
        )

Loading cached processed dataset at /home/akihiczak/.cache/huggingface/datasets/csv/default-3a2c74c780365b63/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-ab157915460051ef.arrow


In [15]:
test_dataset = test_dataset.map(
            preprocess_function,
            batched=True,
            remove_columns=column_names,
        )

Loading cached processed dataset at /home/akihiczak/.cache/huggingface/datasets/csv/default-3a2c74c780365b63/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-b835c47c61565bd7.arrow


In [16]:
def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        
        labels = [[label.strip()] for label in labels]
        return preds, labels

In [17]:
def compute_metrics(eval_preds):
        preds, labels = eval_preds
        
        if isinstance(preds, tuple):
            print('tuple')
            preds = preds[0]
            
            
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        #if data_args.ignore_pad_token_for_loss:
            # Replace -100 in the labels as we can't decode them.
         #   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        
        print(decoded_preds)
        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"bleu": result["score"]}

        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        return result

In [18]:
data_collator = default_data_collator

In [19]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total # of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=25,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    predict_with_generate=True,
    logging_dir='./logs',
    #disable_tqdm=True     #if timing needs to be disabled
)

In [23]:
trainer = Seq2SeqTrainer(
        model=model2,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

In [35]:
trainer.train()

Step,Training Loss
500,2.035100
1000,1.022300
1500,0.944200


TrainOutput(global_step=1875, training_loss=1.2473868326822917, metrics={'train_runtime': 55637.0045, 'train_samples_per_second': 0.034, 'total_flos': 15115450934835000, 'epoch': 5.0})

In [21]:
eval_dataset = test_dataset

In [24]:
metrics = trainer.evaluate(
            max_length=max_target_length, num_beams=1, metric_key_prefix="eval"
        )
max_val_samples = len(eval_dataset)
metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

['[unused2] U S S Navy the of - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -', '[unused2] Korea s of Ministry Defense said Korea s Ministry said Ministry Defense said Ministry said Ministry said', '[unused2] militarys the of Ministry Defense Defense Defense.', '[unused2] Korea s of Ministry Defense said Korea s Ministry Defense s

In [25]:
metrics

{'eval_loss': 1.2720037698745728,
 'eval_bleu': 0.1258,
 'eval_gen_len': 291.5,
 'eval_runtime': 126.5894,
 'eval_samples_per_second': 0.032,
 'eval_samples': 4}

In [38]:
model.save_pretrained('./backups/saved_model_apr6/')

In [20]:
from transformers import AutoModelForSeq2SeqLM
model2 = AutoModelForSeq2SeqLM.from_pretrained('./backups/saved_model_apr6/')

In [22]:
model2

EncoderDecoderModel(
  (encoder): BertGenerationEncoder(
    (embeddings): BertGenerationEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (